In [1]:
# package import
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

In [6]:
def read_csv_file(filename='/Users/yongsukjang/Desktop/kaggle/crypto_forecasting/g-research-crypto-forecasting/train.csv'):
    df = pd.read_csv(filename)
    df['datetime'] = pd.to_datetime(df['timestamp'],unit='s')
    df = df[df['datetime'] < '2021-06-13 00:00:00']
    return df


In [16]:
df_train = read_csv_file()

In [17]:
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,datetime
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218,2018-01-01 00:01:00
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399,2018-01-01 00:01:00
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643,2018-01-01 00:01:00
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922,2018-01-01 00:01:00
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264,2018-01-01 00:01:00


In [18]:
df_asset_details = pd.read_csv('/Users/yongsukjang/Desktop/kaggle/crypto_forecasting/g-research-crypto-forecasting/asset_details.csv').sort_values("Asset_ID")
df_asset_details

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


In [19]:
#Training

In [20]:
#new feature
#upper/lower shadow

def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'],df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'],df['Open']) - df['Low']

In [23]:
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['upper_shadow'] = upper_shadow(df=df_feat)
    df_feat['lower_shadow'] = lower_shadow(df=df_feat)
    return df_feat

In [25]:
get_features(df=df_train).head()

,Count,Open,High,Low,Close,Volume,VWAP,upper_shadow,lower_shadow
0,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,22.920,17.4500
1,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,0.000,0.0000
2,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,163.624,169.0840
3,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,0.000,0.0009
4,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,0.000,0.0030


In [26]:
def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train['Asset_ID'] == asset_id]

    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.replace([np.inf, -np.inf], np.nan).dropna(how='any')

    X=df_proc.drop('y',axis=1)
    y=df_proc['y']

    model = LGBMRegressor(n_estimators=10)
    model.fit(X,y)
    return X, y, model

In [27]:
#Loop over all assets
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)    
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

Training model for Binance Coin     (ID=0 )
Training model for Bitcoin          (ID=1 )
Training model for Bitcoin Cash     (ID=2 )
Training model for Cardano          (ID=3 )
Training model for Dogecoin         (ID=4 )
Training model for EOS.IO           (ID=5 )
Training model for Ethereum         (ID=6 )
Training model for Ethereum Classic (ID=7 )
Training model for IOTA             (ID=8 )
Training model for Litecoin         (ID=9 )
Training model for Maker            (ID=10)
Training model for Monero           (ID=11)
Training model for Stellar          (ID=12)
Training model for TRON             (ID=13)


In [28]:
# Check the model interface
x = get_features(df_train.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

/usr/local/Caskroom/miniforge/base/envs/dacon_cc/lib/python3.8/site-packages/lightgbm/sklearn.py:719: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = _LGBMCheckArray(X, accept_sparse=True, force_all_finite=False)


9.477576309595709e-06